<a href="https://colab.research.google.com/github/marioschlosser/roadmap-analyzer/blob/main/Roadmap_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib google-genai

In [ ]:
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from google.auth.transport.requests import AuthorizedSession
import pathlib
import re
from datetime import datetime, timedelta, timezone
from urllib.parse import urlsplit, urlunsplit, parse_qsl, urlencode

def ensure_gid(url: str, new_gid: int) -> str:
    """
    Return *url* with the query parameter gid=new_gid.
    • Replaces an existing gid=… if present
    • Adds ?gid=… (or &gid=…) if missing
    """
    parts = list(urlsplit(url))          # scheme, netloc, path, query, fragment
    qry_dict = dict(parse_qsl(parts[3], keep_blank_values=True))  # {'format':'csv', ...}

    qry_dict['gid'] = str(new_gid)       # add or overwrite
    parts[3] = urlencode(qry_dict, doseq=True)

    return urlunsplit(parts)

def rfc3339_to_dt(s):
    """Drive returns Z-suffix RFC 3339 → convert to aware datetime."""
    return datetime.fromisoformat(s.replace("Z", "+00:00"))

auth.authenticate_user()
creds, _ = default(scopes=[
    "https://www.googleapis.com/auth/drive.readonly"
])

spreadsheet_id = "17Ml7a1Quk4zmBV6XR9IVnNuBhTs6UaEjHndUhpPHLbk" # @param {"type":"string"}
days_in_past = 14 # @param {"type":"integer"}
worksheet_name = "Tracker" # @param {"type":"string"}
google_api_key = "" # @param {"type":"string"}

drive = build("drive", "v3", credentials=creds)
sheets = build("sheets",  "v4", credentials=creds)

FILE_ID = spreadsheet_id
MIME_XLSX = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"

# --- 0. List all worksheets in the file ----------
sheets_meta = sheets.spreadsheets().get(
    spreadsheetId=FILE_ID,
    fields="sheets(properties(sheetId,title))"
).execute()

name_to_gid = {s['properties']['title']: s['properties']['sheetId']
               for s in sheets_meta['sheets']}

# --- 1. Find the revision closest to 2 weeks ago ----------
target = datetime.now(timezone.utc) - timedelta(days=days_in_past)

rev_items = drive.revisions().list(
    fileId=FILE_ID,
    fields="revisions(id, modifiedTime)"
).execute()["revisions"]

closest = min(
    rev_items,
    key=lambda r: abs(rfc3339_to_dt(r["modifiedTime"]) - target)
)

revs = drive.revisions().list(fileId=FILE_ID, pageSize=100).execute()['revisions']

# Get the revision IDs for the previous and the current versions
rev_id_old = closest["id"]
rev_id_current = revs[-1]["id"]

# Get the worksheet ID
wanted_gid = name_to_gid[worksheet_name]

def get_sheet(rev_id, wanted_gid):
    export_url = drive.revisions().get(
        fileId=FILE_ID,
        revisionId=rev_id,
        fields="exportLinks"
    ).execute()
    csv_base = export_url["exportLinks"]["text/csv"]
    csv_url = ensure_gid(csv_base, new_gid=wanted_gid)
    csv_bytes = AuthorizedSession(creds).get(csv_url).content
    return csv_bytes

# --- 2. Get each revision’s data -------------------
csv_bytes_old = get_sheet(rev_id_old, wanted_gid)
csv_bytes_current = get_sheet(rev_id_current, wanted_gid)

In [ ]:
import base64
import os
from google import genai
from google.genai import types

def generate():
    client = genai.Client(
        api_key=google_api_key,
    )

    model = "gemini-2.5-pro-preview-03-25"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="You run the technology department at Oscar Health. Below are two snapshots from the technology roadmap. Write a summary of where the team has made progress, and where projects haven't progressed."),
            ],
        ),
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=f"From {days_in_past} days ago: {csv_bytes_old}"),
            ],
        ),
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=f"From today: {csv_bytes_current}"),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_mime_type="text/plain",
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

generate()

Okay team, here’s a quick summary comparing our AI/ML technology roadmap from two weeks ago to today. This highlights where we've seen movement and where things have remained static.

**Progress Highlights:**

1.  **New Initiatives Kicked Off:** We've added several new projects to the roadmap, indicating expansion and new areas of exploration:
    *   **Contract Extraction (Claim Dispute Timeline):** Now in "Scoping / business case development" under UM/Clinical.
    *   **Member Issue Analysis (Escalation Care Team):** A new line item under MemEx, though status is currently blank, implying early stages.
    *   **SuperAgent Enhancements:** Three new initiatives leveraging SuperAgent are now in "Prototyping": Member call transcript analysis, Improved Verint search, and LLM enabled quality assessment.

2.  **Quantified Impact & Clarifications:** Several projects, particularly those already launched or in productionalization, have seen updates:
    *   **Impact Data Added:** We've added 